In [ ]:
%pwd

'c:\\Users\\manav\\OneDrive\\Desktop\\Medi Bot\\research'

In [4]:
import os
os.chdir("../")

In [2]:
%pwd

'c:\\Users\\manav\\OneDrive\\Desktop\\Medi Bot'

In [67]:
from dotenv import load_dotenv
import os

load_dotenv()  # This reads your .env file
os.environ["MISTRAL_API_KEY"] = "X5eabql772ixM3WYt0T3EGgQgE4aQTK6"
print(os.getenv("MISTRAL_API_KEY"))  # Just to verify


X5eabql772ixM3WYt0T3EGgQgE4aQTK6


In [ ]:
import langchain
print(langchain.__version__)


1.0.1


In [7]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

c:\Users\manav\anaconda3\envs\mediBot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents


In [9]:
extracted_data=load_pdf_file(data='Data/')

In [9]:
# extracted_data

In [10]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [11]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5859


In [12]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [13]:
#Download the Embeddings from Hugging Face
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
def download_hugging_face_embeddings():
        embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
        return embeddings
    


In [14]:
embeddings = download_hugging_face_embeddings()

C:\Users\manav\AppData\Local\Temp\ipykernel_22628\4267454138.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [15]:
query_result = embeddings.embed_query("Hello world!")
print("Length", len(query_result))

Length 768


In [19]:
# query_result

In [16]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

vector_store.add_documents(text_chunks)

In [83]:
# !pip check
!pip freeze > requirements.txt
# !pip uninstall pinecone-client pinecone-plugin-assistant -y
# !pip install "pinecone[grpc]"

In [26]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [32]:
query = "What is acne?"
retrieved_docs = retriever.invoke(query)

In [33]:
retrieved_docs

[Document(id='285dd3c9-61f0-4542-ac7b-bc556f9829a3', metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637, 'page': 37, 'page_label': '38'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged with oil, dead skin\ncells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne, is\nthe most common skin disease. It affects nearly 17 million\npeople in the United States. While acne can arise at any'),
 Document(id='4559a90b-ebae-4c80-85b3-659cbc9f2b5a', metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'Data

In [74]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(
    model="mistral-large-latest",
    temperature=0.3,
    max_retries=2,
    # other params...
)

print(llm.invoke("What is Acromegaly?"))

content='**Acromegaly** is a rare hormonal disorder that occurs when the **pituitary gland** (a small gland at the base of the brain) produces **excess growth hormone (GH)** during adulthood. This leads to the overproduction of **insulin-like growth factor-1 (IGF-1)**, causing gradual **enlargement of bones and tissues**, particularly in the hands, feet, and face.\n\n### **Causes**\nThe most common cause is a **benign (non-cancerous) tumor (adenoma)** of the pituitary gland that secretes excess GH. Rarely, it can be caused by tumors in other parts of the body (e.g., lungs, pancreas) that produce **growth hormone-releasing hormone (GHRH)**, stimulating the pituitary to overproduce GH.\n\n### **Symptoms**\nAcromegaly develops **slowly over years**, making early diagnosis challenging. Common signs and symptoms include:\n\n#### **Physical Changes:**\n- **Enlarged hands and feet** (rings and shoes no longer fit)\n- **Coarsened facial features** (protruding jaw, enlarged nose, thickened lips

In [35]:
# from langchain.chains import create_retrieval_chain
# from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [75]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "input": RunnablePassthrough()}
    | prompt
    | llm
)

In [76]:
import getpass

if "MISTRAL_API_KEY" not in os.environ:
    os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter your Mistral API key: ")

In [71]:
!echo %MISTRAL_API_KEY%

X5eabql772ixM3WYt0T3EGgQgE4aQTK6


In [ ]:
response = rag_chain.invoke("what is Acromegaly and gigantism?")
print(response.content)

Acromegaly is a disorder caused by the abnormal release of growth hormone from the pituitary gland, leading to increased growth in bones and soft tissues, as well as other bodily disturbances.

Gigantism is a related condition that occurs when excess growth hormone is released **before** the growth plates in bones close (typically during childhood), resulting in abnormal height and size. Both conditions stem from growth hormone excess but differ in timing and effects.


In [82]:
response = rag_chain.invoke("What is stats?")
print(response.content)

I don't know. The provided context does not contain information about what "stats" refers to.
